In [4]:
import pandas as pd

# Caminho do arquivo CSV
caminho_arquivo = r"C:\Users\rafae\Desktop\clinica_sevem\Data Lake\orders_raw.csv"

# Leitura do arquivo CSV
df = pd.read_csv(caminho_arquivo)

df


,order_id,var_name,var_value
0,order_1,user_id,user_270
1,order_1,created_at,2023-09-09
2,order_1,items,"[{'product_id': 'prod_77', 'quantity': 3}, {'p..."
3,order_1,total,"R$ 2,393,37"
4,order_1,payment_status,pago
...,...,...,...
2695,order_300,payment_status,não pago
2696,order_300,payment_method,NaN
2697,order_300,payment_date,NaN
2698,order_300,shipping_status,enviado


In [5]:
print(df['var_name'].unique())


['user_id' 'created_at' 'items' 'total' 'payment_status' 'payment_method'
 'payment_date' 'shipping_status' 'shipping_status_date']


In [6]:
df_wide = df.pivot(index='order_id', columns='var_name', values='var_value').reset_index()
df_wide.head(1)

var_name,order_id,created_at,items,payment_date,payment_method,payment_status,shipping_status,shipping_status_date,total,user_id
0,order_1,2023-09-09,"[{'product_id': 'prod_77', 'quantity': 3}, {'p...",2023-01-11,boleto,pago,enviado,2023/05/27,"R$ 2,393,37",user_270


In [7]:
import pandas as pd
import ast

# Supondo que df_wide já esteja carregado
df_explode = []

for _, row in df_wide.iterrows():
    try:
        # Tenta converter a string em lista de dicionários
        items = ast.literal_eval(row['items']) if pd.notnull(row['items']) else []
        
        # Se for lista de dicionários, cria nova linha por item
        for item in items:
            nova_linha = row.copy()
            nova_linha['product_id'] = item.get('product_id')
            nova_linha['quantity'] = item.get('quantity')
            df_explode.append(nova_linha)
    
    except (ValueError, SyntaxError) as e:
        # Caso falhe na conversão, apenas pula a linha
        print(f"Erro ao processar linha com order_id={row['order_id']}: {e}")

In [8]:


# Novo DataFrame com os itens explodidos
df_final = pd.DataFrame(df_explode)

df_final.head(1)

var_name,order_id,created_at,items,payment_date,payment_method,payment_status,shipping_status,shipping_status_date,total,user_id,product_id,quantity
0,order_1,2023-09-09,"[{'product_id': 'prod_77', 'quantity': 3}, {'p...",2023-01-11,boleto,pago,enviado,2023/05/27,"R$ 2,393,37",user_270,prod_77,3


In [9]:
# #verifica o q esta dentro da coluna itens
# display(df_final['items'].unique())


In [10]:
df_final.drop(columns=['items'], inplace=True)


In [11]:
df_final.head(1)

var_name,order_id,created_at,payment_date,payment_method,payment_status,shipping_status,shipping_status_date,total,user_id,product_id,quantity
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023/05/27,"R$ 2,393,37",user_270,prod_77,3


In [12]:
print(df_final.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   order_id              753 non-null    object
 1   created_at            753 non-null    object
 2   payment_date          270 non-null    object
 3   payment_method        270 non-null    object
 4   payment_status        753 non-null    object
 5   shipping_status       753 non-null    object
 6   shipping_status_date  753 non-null    object
 7   total                 753 non-null    object
 8   user_id               753 non-null    object
 9   product_id            753 non-null    object
 10  quantity              753 non-null    int64 
dtypes: int64(1), object(10)
memory usage: 70.6+ KB
None


In [13]:
colunas_data = ['created_at', 'payment_date', 'shipping_status_date']
for col in colunas_data:
    df_final[col] = pd.to_datetime(df_final[col], errors='coerce', dayfirst=True)


In [14]:
df_final.head(1)

var_name,order_id,created_at,payment_date,payment_method,payment_status,shipping_status,shipping_status_date,total,user_id,product_id,quantity
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023-05-27,"R$ 2,393,37",user_270,prod_77,3


In [15]:
print(df_final.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              753 non-null    object        
 1   created_at            753 non-null    datetime64[ns]
 2   payment_date          270 non-null    datetime64[ns]
 3   payment_method        270 non-null    object        
 4   payment_status        753 non-null    object        
 5   shipping_status       753 non-null    object        
 6   shipping_status_date  753 non-null    datetime64[ns]
 7   total                 753 non-null    object        
 8   user_id               753 non-null    object        
 9   product_id            753 non-null    object        
 10  quantity              753 non-null    int64         
dtypes: datetime64[ns](3), int64(1), object(7)
memory usage: 70.6+ KB
None


In [17]:
# 2. Limpar coluna "total" e converter para float
df_final['total'].replace('R\$', '', regex=True, inplace=True)       # Remove "R$"
df_final['total'].replace('\.', '', regex=True, inplace=True)        # Remove ponto (milhar)
df_final['total'].replace(',', '.', regex=True, inplace=True)        # Troca vírgula decimal por ponto
df_final['total'] = df_final['total'].astype(float)   

In [18]:
print(df_final.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 753 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   order_id              753 non-null    object        
 1   created_at            753 non-null    datetime64[ns]
 2   payment_date          270 non-null    datetime64[ns]
 3   payment_method        270 non-null    object        
 4   payment_status        753 non-null    object        
 5   shipping_status       753 non-null    object        
 6   shipping_status_date  753 non-null    datetime64[ns]
 7   total                 753 non-null    float64       
 8   user_id               753 non-null    object        
 9   product_id            753 non-null    object        
 10  quantity              753 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(1), object(6)
memory usage: 70.6+ KB
None


In [19]:
df_final.head(100)

var_name,order_id,created_at,payment_date,payment_method,payment_status,shipping_status,shipping_status_date,total,user_id,product_id,quantity
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023-05-27,239337.0,user_270,prod_77,3
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023-05-27,239337.0,user_270,prod_39,1
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023-05-27,239337.0,user_270,prod_58,2
0,order_1,2023-09-09,2023-01-11,boleto,pago,enviado,2023-05-27,239337.0,user_270,prod_26,1
1,order_10,2023-06-26,NaT,NaN,pendente,enviado,2023-05-24,247036.0,user_746,prod_93,2
...,...,...,...,...,...,...,...,...,...,...,...
37,order_132,2023-05-12,NaT,NaN,não pago,entregue,2023-10-30,228571.0,user_989,prod_50,3
37,order_132,2023-05-12,NaT,NaN,não pago,entregue,2023-10-30,228571.0,user_989,prod_82,2
37,order_132,2023-05-12,NaT,NaN,não pago,entregue,2023-10-30,228571.0,user_989,prod_85,2
38,order_133,2023-02-11,2023-12-04,débito,pago,preparando envio,2023-07-15,241481.0,user_799,prod_37,2


In [20]:
df_final.to_csv("pedidos_tratado.csv", index=False)
